# method 1
from github: jalajthanaki/stock_price_prediction
<br><br>
06282020: error, can not find newsapi.articles


In [4]:
"""
About:
Python wrapper for the New York Times Archive API
https://developer.nytimes.com/article_search_v2.json
"""
from newsapi.articles import Articles
import sys,json
import requests

key = '522497f7b4b940b7946eeed6909ed817'
params = {}
api = Articles(API_KEY=key)
reload(sys)
sys.setdefaultencoding('utf8')


class APIKeyException(Exception):
    def __init__(self, message): self.message = message


class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message


class ArchiveAPI(object):
    def __init__(self, key=None):
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise APIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None, ):
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1851
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        #print url ##boting: remove error message
        r = requests.get(url)
        return r.json()




ModuleNotFoundError: No module named 'newsapi.articles'

In [5]:
import newsapi 

In [8]:
from newsapi.articles import Articles

ModuleNotFoundError: No module named 'newsapi.articles'

In [7]:
dir(newsapi)

['NewsApiClient',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'const',
 'newsapi_auth',
 'newsapi_client',
 'newsapi_exception',
 'utils']

In [ ]:
years = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
api = ArchiveAPI('522497f7b4b940b7946eeed6909ed817')
for year in years:
    for month in months:
        mydict = api.query(year, month)
        file_str = './data/nytimes' + str(year) + '-' + '{:02}'.format(
            month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()

# method 2
from github: jasonyip184/StockSentimentTrading
 
business times

In [14]:
import nltk
nltk.download('vader_lexicon')

import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /Users/wangbo-
[nltk_data]     ting/nltk_data...


In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

29 Jun 2020 https://www.businesstimes.com.sg/technology/facebook-ad-boycott-campaign-to-go-global-organisers-say
27 Jun 2020 https://www.businesstimes.com.sg/technology/unilever-to-stop-advertising-on-facebook-twitter-and-instagram-in-us
27 Jun 2020 https://www.businesstimes.com.sg/technology/facebook-ramps-up-efforts-to-curb-hateful-content-in-ads
26 Jun 2020 https://www.businesstimes.com.sg/technology/verizon-suspends-advertising-on-facebook-joins-growing-boycott
24 Jun 2020 https://www.businesstimes.com.sg/consumer/ben-jerrys-joins-facebook-ad-boycott-over-hate-speech
23 Jun 2020 https://www.businesstimes.com.sg/government-economy/biden-campaign-asks-twitter-facebook-to-remove-trump-posts-bashing-mail-in-voting
23 Jun 2020 https://www.businesstimes.com.sg/technology/microsoft-ends-game-streaming-teams-up-with-facebook
19 Jun 2020 https://www.businesstimes.com.sg/government-economy/facebook-pulls-trump-ads-over-nazi-symbol
18 Jun 2020 https://www.businesstimes.com.sg/technology/faceb

13 Feb 2020 https://www.businesstimes.com.sg/technology/facebook-owned-whatsapp-says-it-has-2-billion-users
08 Feb 2020 https://www.businesstimes.com.sg/technology/twitter-says-facebook-messenger-accounts-hacked
31 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-to-remove-virus-misinformation-after-who-declares-global-emergency
31 Jan 2020 https://www.businesstimes.com.sg/technology/facebooks-revenue-rises-again-but-more-slowly-than-ever
30 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-results-beat-forecasts-but-shares-take-a-hit
29 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-rolls-out-tool-globally-to-clear-third-party-data
28 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-asks-employees-to-put-off-non-essential-travel-to-china
23 Jan 2020 https://www.businesstimes.com.sg/banking-finance/facebook-spurred-central-banks-to-study-digital-currencies-ex-boj-exec
22 Jan 2020 https://www.businesstimes.com.sg/banking-finance/vodaf

In [17]:
date_sentiment

18 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-must-disclose-app-records-for-probe-us-judge-rules
 Hello, your email is unverified. Please confirm  
0.9844
10 Jan 2020 https://www.businesstimes.com.sg/technology/lawmakers-facebooks-steps-to-tackle-deepfake-videos-inadequate
 Hello, your email is unverified. Please confirm  
-0.8632
08 Jan 2020 https://www.businesstimes.com.sg/consumer/facebooks-new-singapore-team-to-serve-china-advertising-business
 Hello, your email is unverified. Please confirm  
0.995
08 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-tightening-policy-against-deepfakes
 Hello, your email is unverified. Please confirm  
0.9126
07 Jan 2020 https://www.businesstimes.com.sg/government-economy/trump-says-zuckerberg-told-him-hes-facebooks-number-one
 Hello, your email is unverified. Please confirm  
-0.628
28 Dec 2019 https://www.businesstimes.com.sg/banking-finance/facebooks-libra-has-failed-in-current-form-says-swiss-president
 Hello, yo

In [18]:
date_sentiment

NameError: name 'date_sentiment' is not defined

# method 3
from github dineshdaultani/StockPredictions

NYTimes

need an api 'key'

In [22]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 20 00:10:29 2017
@author: Dinesh
"""
# Below part is not required
########## News API ########################################################
# from newsapi import NewsAPI

# key = '96af62a035db45bda517a9ca62a25ac3'
# params = {}
# api = NewsAPI(key)
# sources = api.sources(params)
# articles = api.articles(sources[0]['id'], params)

################ NY Times API #############################################
import sys, csv, json
#reload(sys) #boting: reload has been removed since python 3.4
#sys.setdefaultencoding('utf8') #boting: sys.setdefaultencoding is not recommended for use


import requests
"""
About:
Python wrapper for the New York Times Archive API 
https://developer.nytimes.com/article_search_v2.json
"""

class APIKeyException(Exception):
    def __init__(self, message): self.message = message 

class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message 

class ArchiveAPI(object):
    def __init__(self, key=None):
        """
        Initializes the ArchiveAPI class. Raises an exception if no API key is given.
        :param key: New York Times API Key
        """
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}' 
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise NoAPIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None,):
        """
        Calls the archive API and returns the results as a dictionary.
        :param key: Defaults to the API key used to initialize the ArchiveAPI class.
        """
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1882
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        r = requests.get(url)
        return r.json()


        

    

In [26]:
import os

In [27]:
# Replace below key with your NYTimes Developer key
api = ArchiveAPI('key')

#years = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
years = [2016]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for year in years:
    for month in months:
        mydict = api.query(year, month)
        #file_str = '/Users/Dinesh/Documents/Project Stock predictions/data/nytimes/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        if not os.path.exists('./news/nytimes/'):
            os.makedirs('./news/nytimes/')
        file_str = './news/nytimes/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()

# method 4
from github: WayneDW/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction

# method 5
github: Keess324/News_predict_Stock_Price_trend


In [10]:
# Import Library
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
#from datetime import datetime
from pandas_datareader import data as web
import os
import string
import requests
import nltk
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader

In [11]:
# Stock Price
def stock(stock_type, start, end):
    sp500stock = web.get_data_yahoo([stock_type], start,end)
    return sp500stock
    
stock_type = '^GSPC'
start=datetime.datetime(2016, 9, 26)
end=datetime.datetime(2018,10, 1)

stock_data = stock(stock_type, start, end)

In [12]:
stock_data.head(10)


Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
2016-09-26,2146.100098,2146.100098,2158.540039,2145.040039,2158.540039,3216170000
2016-09-27,2159.929932,2159.929932,2161.129883,2141.550049,2146.040039,3437770000
2016-09-28,2171.370117,2171.370117,2172.399902,2151.790039,2161.850098,3891460000
2016-09-29,2151.129883,2151.129883,2172.669922,2145.199951,2168.899902,4249220000
2016-09-30,2168.270020,2168.270020,2175.300049,2156.510010,2156.510010,4173340000
2016-10-03,2161.199951,2161.199951,2164.409912,2154.770020,2164.330078,3137550000
2016-10-04,2150.489990,2150.489990,2165.459961,2144.010010,2163.370117,3750890000
2016-10-05,2159.729980,2159.729980,2163.949951,2155.149902,2155.149902,3906550000


In [13]:
open_date = stock_data.index
len(open_date)

508

In [14]:
#Scraping news from the the corresponding date we found
date_loop_list = []
for i in range(len(open_date)):
    d1 = open_date[i]
    date_loop_list.append(d1.date())
print(len(date_loop_list))


508


In [22]:
#date_loop_list
date_loop_list[0]

datetime.date(2016, 9, 26)

In [ ]:
#if error message when running BeautifulSoup(page.content,'lxml'):
#FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
#you need to install lxml in the command line
#pip3 install lxml

#after installing it, it still does not work,
#but BeautifulSoup(page.content,"html.parser")


In [33]:
date = date_loop_list[0]
url = 'http://www.wsj.com/public/page/archive-'+ str(date) + '.html'
page = requests.get(url)    
#result = BeautifulSoup(page.content,"lxml")


In [36]:
result = BeautifulSoup(page.content,"html.parser")

In [37]:
result


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<style>@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-Book.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-Book.woff) format("woff");font-weight:400;font-style:normal;font-display:swap}@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-BookItalic.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-BookItalic.woff) format("woff");font-weight:400;font-style:italic;font-display:optional}@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-Light.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-Light.woff) format("woff");font-weight:300;font-style:normal;font-display:swap}@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-LightItalic.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-LightItalic.woff) fo

In [38]:
# Web Scraping from Wall Street Journal
def news(dateloop):
#     date_list = [base - datetime.timedelta(days=x) for x in range(daterange)]
#     date_loop_list = []
#     for i in date_list:
#         dt = datetime.date.strftime(i, '%Y-%m-%d')
#         date_loop_list.append(dt)
    
    All_News=[]
    for date in dateloop:
        url = 'http://www.wsj.com/public/page/archive-'+ str(date) + '.html'
        page = requests.get(url)    
        result = BeautifulSoup(page.content,'lxml')
        UL = result.find_all('ul', class_ = 'newsItem')
        body1 = UL[0].find_all('p')
        everyday_results = []
        Headlines_Day = []
        Abstracts_Day = []
        for news_ in range(len(body1)):
            body2 = body1[news_].get_text()
            body3 = body2.strip().split('\n ')
            Headlines_item = body3[0]
            Abstracts_item = body3[-1]
            Headlines_Day.append(Headlines_item)
            Abstracts_Day.append(Abstracts_item)
        everyday_results = [date, Headlines_Day, Abstracts_Day]
        All_News.append(everyday_results)
    return All_News

In [39]:
output_news = news(date_loop_list)
# output_news

IndexError: list index out of range

In [17]:
def news_to_dataframe(output_news):
    news_df = pd.DataFrame()
    date = []
    news_title = []
    news_abstract = []
    for day in output_news:
        date.append(day[0])  # date
        news_title.append(day[1]) # news_title
        news_abstract.append(day[2]) # news_abstract
    news_df['Date'] = date
    news_df['News_title'] = news_title
    news_df['News_abstract'] = news_abstract
    
    return news_df


In [18]:
news_data = news_to_dataframe(output_news) 
news_data.head(10)

NameError: name 'output_news' is not defined

# method 6
github: WayneDW/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction

seems can not download ./input/tickerList.csv

In [1]:
#!/usr/bin/env python3
"""
Download the ticker list from NASDAQ and save as csv.
Output filename: ./input/tickerList.csv
"""
import csv
import sys

from urllib.request import urlopen

import numpy as np


def get_tickers(percent):
    """Keep the top percent market-cap companies."""
    assert isinstance(percent, int)

    file = open('./input/tickerList.csv', 'w')
    writer = csv.writer(file, delimiter=',')
    cap_stat, output = np.array([]), []
    for exchange in ["NASDAQ", "NYSE", "AMEX"]:
        url = "http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange="
        repeat_times = 10 # repeat downloading in case of http error
        for _ in range(repeat_times):
            try:
                print("Downloading tickers from {}...".format(exchange))
                response = urlopen(url + exchange + '&render=download')
                print('finish urlopen')
                content = response.read().decode('utf-8').split('\n')
                print('finish content')
                for num, line in enumerate(content):
                    print(num) #boting
                    line = line.strip().strip('"').split('","')
                    if num == 0 or len(line) != 9:
                        continue # filter unmatched format
                    # ticker, name, last_sale, market_cap, IPO_year, sector, industry
                    ticker, name, _, market_cap, _, _, _ = line[0:4] + line[5:8]
                    cap_stat = np.append(cap_stat, float(market_cap))
                    output.append([ticker, name.replace(',', '').replace('.', ''),
                                   exchange, market_cap])
                break
            except:
                continue

    for data in output:
        market_cap = float(data[3])
        if market_cap < np.percentile(cap_stat, 100 - percent):
            continue
        writer.writerow(data)


def main():
    if len(sys.argv) < 2:
        print('Usage: ./all_tickers.py <int_percent>')
        return
    top_n = sys.argv[1]
    get_tickers(int(top_n)) # keep the top N% market-cap companies


#if __name__ == "__main__":
#    main()

In [ ]:
import os
#set current directory to test the code
os.chdir('/Users/wangbo-ting/git/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/')
top_n = 20
get_tickers(int(top_n))

In [3]:
import os

isFile = os.path.isfile('../Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/input/tickerList.csv')  
print(isFile) 

True


In [16]:
isDir = os.path.isdir('../Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction')  
print(isDir) 

True


In [9]:
os. getcwd()

'/Users/wangbo-ting/git/sentiment_analysis_stock'

In [3]:
url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download' 

In [ ]:
response = urlopen(url)

In [ ]:
urllib